This notebook plots the HSC luminosity functions (from https://arxiv.org/abs/2108.01090), and calculates numbers for LSST.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18
from astropy import units as u
from photerr import LsstErrorModel

In [ ]:
def dpl_lf(M, params):
    """A Double Power Law Luminosity function"""
    norm = np.log(10) / 2.5 * (10 ** params["norm"])
    dpl = (
        10 ** ((1 + params["alpha"]) / 2.5 * (M - params["Mstar"])) + 
        10 ** ((1 + params["beta"]) / 2.5 * (M - params["Mstar"]))
    )
    return norm / dpl

In [ ]:
lf_params = {
    4: {
        "norm": -3.09,
        "Mstar": -21.10,
        "alpha": -1.87,
        "beta": -4.95,
    },
    5: {
        "norm": -3.48,
        "Mstar": -21.39,
        "alpha": -1.94,
        "beta": -4.96,
    },
    6: {
        "norm": -3.67,
        "Mstar": -21.23,
        "alpha": -2.14,
        "beta": -5.03,
    },
    7: {
        "norm": -3.51,
        "Mstar": -20.82,
        "alpha": -2.05,
        "beta": -4.83,
    },
}

In [ ]:
hsc = {
    4: {
        "mags": [-24.39, -24.09, -23.79, -23.49, -23.19, -22.89, -22.64, -22.44, -22.29, -22.19, -22.09, -21.99, -21.89, -21.79, -21.69, -21.59, -21.49, -21.39, -21.29, -21.19, -21.09, -20.99, -20.89, -20.79, -20.69, -20.59, -20.49, -20.39, -20.29, -20.19, -20.09, -19.99],
        "lf": [1.11e-8, 4.30e-8, 9.22e-8, 2.17e-7, 3.63e-7, 8.07e-7, 1.97e-6, 5.67e-6, 9.24e-6, 1.53e-5, 1.89e-5, 2.75e-5, 3.66e-5, 5.44e-5, 7.38e-5, 1.03e-4, 1.28e-4, 1.82e-4, 2.23e-4, 2.92e-4, 3.86e-4, 4.58e-4, 5.16e-4, 6.35e-4, 8.15e-4, 9.08e-4, 1.03e-3, 1.19e-3, 1.46e-3, 1.59e-3, 1.78e-3, 2.02e-3],
        "err+": [2.74e-8, 5.65e-8, 9.32e-8, 1.83e-7, 2.30e-7, 4.28e-7, 0.92e-6, 2.01e-6, 4.10e-6, 0.61e-5, 0.68e-5, 0.92e-5, 1.13e-5, 1.58e-5, 1.89e-5, 0.23e-4, 0.26e-4, 0.34e-4, 0.38e-4, 0.49e-4, 0.67e-4, 0.81e-4, 0.93e-4, 1.16e-4, 1.51e-4, 1.72e-4, 0.20e-3, 0.24e-3, 0.31e-3, 0.36e-3, 0.44e-3, 0.54e-3],
        "err-": [1.11e-8, 4.30e-8, 9.22e-8, 1.83e-7, 2.30e-7, 4.28e-7, 0.92e-6, 2.01e-6, 4.10e-6, 0.61e-5, 0.68e-5, 0.92e-5, 1.13e-5, 1.58e-5, 1.89e-5, 0.23e-4, 0.26e-4, 0.34e-4, 0.38e-4, 0.49e-4, 0.67e-4, 0.81e-4, 0.93e-4, 1.16e-4, 1.51e-4, 1.72e-4, 0.20e-3, 0.24e-3, 0.31e-3, 0.36e-3, 0.44e-3, 0.54e-3],
    },
    5: {
        "mags": [-24.42, -23.92, -23.42, -23.05, -22.80, -22.55, -22.30, -22.05, -21.80, -21.55, -21.30, -21.05, -20.80, -20.55, -20.30],
        "lf": [2.58e-9, 4.73e-8, 1.57e-7, 6.22e-7, 1.27e-6, 5.21e-6, 1.24e-5, 2.97e-5, 4.78e-5, 1.00e-4, 1.77e-4, 2.93e-4, 4.09e-4, 4.31e-4, 6.13e-4],
        "err+": [14.68e-9, 10.22e-8, 2.48e-7, 4.19e-7, 0.67e-6, 2.28e-6, 0.43e-5, 0.80e-5, 1.10e-5, 0.20e-4, 0.38e-4, 0.71e-4, 1.14e-4, 1.50e-4, 2.52e-4],
        "err-": [2.58e-9, 4.73e-8, 1.57e-7, 4.19e-7, 0.66e-6, 2.28e-6, 0.43e-5, 0.80e-5, 1.10e-5, 0.20e-4, 0.38e-4, 0.71e-4, 1.14e-4, 1.50e-4, 2.52e-4],
    },
    6: {
        "mags": [-24.02, -23.52, -23.12, -22.82, -22.52, -22.22, -21.92, -21.62, -21.32, -21.02],
        "lf": [6.00e-9, 3.76e-8, 1.80e-7, 7.59e-7, 1.81e-6, 3.46e-6, 9.58e-6, 3.55e-5, 7.35e-5, 1.77e-4],
        "err+": [9.95e-9, 2.97e-8, 1.09e-7, 4.14e-7, 0.96e-6, 1.78e-6, 4.82e-6, 1.80e-5, 4.05e-5, 1.22e-4],
        "err-": [6.00e-9, 2.05e-8, 1.09e-7, 4.14e-7, 0.96e-6, 1.78e-6, 4.82e-6, 1.80e-5, 4.05e-5, 1.22e-4],
    },
    7: {
        "mags": [-24.42, -23.92, -23.42, -22.92, -22.42, -21.92],
        "lf": [5.00e-10, 1.31e-8, 4.39e-8, 1.83e-7, 1.06e-6, 2.75e-6],
        "err+": [24.81e-10, 2.46e-8, 6.02e-8, 3.62e-7, 1.20e-6, 2.77e-6],
        "err-": [5.00e-10, 1.31e-8, 4.39e-8, 1.83e-7, 1.06e-6, 2.55e-6],
    },
}

In [ ]:
hubble = {
    4: {
        "mags": [-22.69, -22.19, -21.69, -21.19, -20.69, -20.19, -19.69, -19.19, -18.69, -17.94, -16.94, -15.94],
        "lf": [5e-6, 1.5e-5, 1.44e-4, 3.44e-4, 6.98e-4, 1.624e-3, 2.276e-3, 3.056e-3, 4.371e-3, 1.0160e-2, 2.7420e-2, 2.8820e-2],
        "err+": [4e-6, 9e-6, 2.2e-5, 3.8e-5, 6.8e-5, 1.31e-4, 1.99e-4, 3.88e-4, 6.89e-4, 9.20e-4, 3.440e-3, 8.740e-3],
        "err-": [4e-6, 9e-6, 2.2e-5, 3.8e-5, 6.8e-5, 1.31e-4, 1.99e-4, 3.88e-4, 6.89e-4, 9.20e-4, 3.440e-3, 8.740e-3],
    },
    5: {
        "mags": [-23.11, -22.61, -22.11, -21.61, -21.11, -20.61, -20.11, -19.61, -19.11, -18.36, -17.36, -16.36],
        "lf": [1e-6, 4e-6, 2.8e-5, 9.2e-5, 2.62e-4, 5.84e-4, 8.79e-4, 1.594e-3, 2.159e-3, 4.620e-3, 8.780e-3, 2.5120e-2],
        "err+": [1e-6, 2e-6, 7e-6, 1.3e-5, 2.4e-5, 4.4e-5, 6.7e-5, 1.56e-4, 3.46e-4, 5.20e-4, 1.540e-3, 7.340e-3],
        "err-": [1e-6, 2e-6, 7e-6, 1.3e-5, 2.4e-5, 4.4e-5, 6.7e-5, 1.56e-4, 3.46e-4, 5.20e-4, 1.540e-3, 7.340e-3],
    },
    6: {
        "mags": [-22.52, -22.02, -21.52, -21.02, -20.52, -20.02, -19.52, -18.77, -17.77, -16.77],
        "lf": [2e-6, 1.4e-5, 5.1e-5, 1.69e-4, 3.17e-4, 7.24e-4, 1.147e-3, 2.820e-3, 8.360e-3, 1.7100e-2],
        "err+": [2e-6, 5e-6, 1.1e-5, 2.4e-5, 4.1e-5, 8.7e-5, 1.57e-4, 4.40e-4, 1.660e-3, 5.260e-3],
        "err-": [2e-6, 5e-6, 1.1e-5, 2.4e-5, 4.1e-5, 8.7e-5, 1.57e-4, 4.40e-4, 1.660e-3, 5.260e-3],
    },
    7: {
        "mags": [-22.19, -21.69, -21.19, -20.69, -20.19, -19.69, -19.19, -18.69, -17.94, -16.94],
        "lf": [1e-6, 4.1e-5, 4.7e-5, 1.98e-4, 2.83e-4, 5.89e-4, 1.172e-3, 1.433e-3, 5.760e-3, 8.320e-3],
        "err+": [2e-6, 1.1e-5, 1.5e-5, 3.6e-5, 6.6e-5, 1.26e-4, 3.36e-4, 4.19e-4, 1.440e-3, 2.900e-3],
        "err-": [2e-6, 1.1e-5, 1.5e-5, 3.6e-5, 6.6e-5, 1.26e-4, 3.36e-4, 4.19e-4, 1.440e-3, 2.900e-3],
    },
}

In [ ]:
def plot_lf(z, lsst=True, deep_time=20, wide_time=2, deep_area=100, wide_area=1000):
    """Plot the luminosity function for the given redshift.
    
    Includes HSC and Hubble data.
    """
    # create the figure
    fig, ax = plt.subplots(figsize=(4.5, 3.3), dpi=120)

    # plot the luminosity function
    # using parameters from GOLDRUSH IV
    M = np.linspace(-27, -15, 100_000)
    lf = dpl_lf(M, lf_params[z])
    ax.plot(M, lf, c="gray", label="Luminosity function")

    # plot the HSC data
    ax.errorbar(
        hsc[z]["mags"], 
        hsc[z]["lf"], 
        yerr=[hsc[z]["err-"], hsc[z]["err+"]], 
        ls="", 
        marker="o", 
        markersize=4,
        capsize=3, 
        c="C0", 
        label="HSC", 
    )

    # plot Hubble data
    ax.errorbar(
        hubble[z]["mags"], 
        hubble[z]["lf"], 
        yerr=[hubble[z]["err-"], hubble[z]["err+"]], 
        ls="", 
        marker="s", 
        markersize=4,
        capsize=3, 
        c="k", 
        label="Hubble", 
    )

    # simulate LSST observations
    # --------------------------
    bin_width = 0.25
    bin_centers = np.arange(-27, -15, bin_width) + bin_width / 2

    # total volume (Mpc^3)
    tot_vol = Planck18.comoving_volume(z + 0.5) - Planck18.comoving_volume(z - 0.5)
    tot_vol = tot_vol.value

    # fraction of the sky
    sky_frac = (wide_area * u.deg**2).to(u.steradian) / (4 * np.pi * u.steradian)

    # observed volume
    obs_vol = sky_frac * tot_vol

    # number of galaxies observed
    binned_lf = dpl_lf(bin_centers, lf_params[z])
    N_galaxies = binned_lf * obs_vol * bin_width

    # above the deep limit, we do not detect any galaxies
    detected_band = {4: "g", 5: "r", 6: "i", 7: "z"}[z]
    deep_model = LsstErrorModel(nYrObs=deep_time)
    deep_limit = (
        deep_model.getLimitingMags()[detected_band]
        + 2.5 * np.log10(1 + z)
        - 5 * np.log10(Planck18.luminosity_distance(z) / (10 * u.pc))
    )
    N_galaxies[bin_centers > deep_limit] *= 0

    # sample from Poisson
    rng = np.random.default_rng(4)
    N_obs = rng.poisson(N_galaxies)

    # calculate observations
    lf_obs = N_obs / obs_vol / bin_width
    errs = np.sqrt(N_obs) / obs_vol / bin_width

    # above the wide limit, we need to use the deep field, which has 10x less area
    # so we magnify the Poisson errors by sqrt(10)
    wide_model = LsstErrorModel(nYrObs=wide_time)
    wide_limit = (
        wide_model.getLimitingMags()[detected_band]
        + 2.5 * np.log10(1 + z)
        - 5 * np.log10(Planck18.luminosity_distance(z) / (10 * u.pc))
    )
    errs[bin_centers > wide_limit] *= np.sqrt(wide_area / deep_area)

    ax.errorbar(
        bin_centers, 
        lf_obs, 
        yerr=errs, 
        ls="", 
        marker="*", 
        markersize=6,
        capsize=3, 
        c="C3", 
        label="LSST Commissioning" if lsst else None,
        alpha=lsst,
    )

    ax.legend(loc="lower right")
    ax.set(
        yscale="log", 
        xlabel="$M_{UV}$", 
        ylabel="$\phi(M_{UV})$  [Mpc$^{-3}$ mag$^{-1}$ ]",
        xlim=(-26, -15.5),
        ylim=(1e-11, 1e-1),
    )
    ax.text(0.05, 0.95, f"redshift ~ {z}", transform=ax.transAxes, va="top", ha="left")

    plt.tight_layout()

    # before we're done, let's print the total number of galaxies!
    # we need to down-weight the number of galaxies in the deep field (when I
    # corrected for the deep field above, I only scaled the error bars)
    N_obs = N_obs.astype(float)
    N_obs[bin_centers > wide_limit] /= wide_area / deep_area

    # finally, let's print the number of galaxies!
    print(f"total galaxies: {N_obs.sum() / 1e6: .2f}e6")

    return fig, ax

### Commissioning plots

In [ ]:
fig, ax = plot_lf(4, lsst=False)
fig.savefig("figures/lf_z=4.png", dpi=500)

In [ ]:
fig, ax = plot_lf(4)
fig.savefig("figures/lf_z=4_LsstComm.png", dpi=500)

In [ ]:
fig, ax = plot_lf(5)
fig.savefig("figures/lf_z=5_LsstComm.png", dpi=500)

In [ ]:
fig, ax = plot_lf(6)
fig.savefig("figures/lf_z=6_LsstComm.png", dpi=500)

In [ ]:
fig, ax = plot_lf(7)
fig.savefig("figures/lf_z=7_LsstComm.png", dpi=500)

### Full survey plots

In [ ]:
fig, ax = plot_lf(4, deep_time=200, wide_time=10, deep_area=40, wide_area=18_000)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig("figures/lf_z=4_LsstFull.png", dpi=500)

In [ ]:
fig, ax = plot_lf(5, deep_time=200, wide_time=10, deep_area=40, wide_area=18_000)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig("figures/lf_z=5_LsstFull.png", dpi=500)

In [ ]:
fig, ax = plot_lf(6, deep_time=200, wide_time=10, deep_area=40, wide_area=18_000)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig("figures/lf_z=6_LsstFull.png", dpi=500)

In [ ]:
fig, ax = plot_lf(7, deep_time=200, wide_time=10, deep_area=40, wide_area=18_000)
ax.set(xlim=(-27, -15.5), ylim=(1e-12, 1e-1))
fig.savefig("figures/lf_z=7_LsstFull.png", dpi=500)